# Data Collection


Here you are asked to collect data to then guess the model of the Duckiebot


In [22]:
from lraClass import LRA2_HELPER

import time
import sys
import os
import argparse
import math
import numpy as np
import gym
from simulator.src.gym_duckietown.envs import DuckietownEnv


Enviroment initialization

In [30]:
lra = LRA2_HELPER()


env = DuckietownEnv(
    map_name = "udem1",
    domain_rand = False,
    draw_bbox = False
)

env = lra.wrapEnv(env)

env.reset()


INFO:gym-duckietown:Information about the graphics card:
 information: dict[4]
              │ vendor: NVIDIA Corporation
              │ renderer: GeForce GTX 1050 Ti with Max-Q Design/PCIe/SSE2
              │ version: 4.6.0 NVIDIA 450.66
              │ shading-language-version: 4.60 NVIDIA
DEBUG:gym-duckietown:loading map file "/home/mike/duckietown/RH9/lqr-exercise/venv/lib/python3.8/site-packages/duckietown_world/data/gd1/maps/udem1.yaml"
INFO:gym-duckietown:done
INFO:gym-duckietown: light_pos: [0.0, 3.0, 0.0, 1.0]
   ambient: instance of ndarray: array([0.25, 0.25, 0.25, 1.  ])
   diffuse: instance of ndarray: array([0.35, 0.35, 0.35, 1.  ])
  specular: instance of ndarray: array([0., 0., 0., 1.])
DEBUG:gym-duckietown:[2.00897243 0.         3.51472137] corresponds to tile at (3, 6) which is not drivable: {'coords': (3, 6), 'kind': 'floor', 'angle': 1, 'drivable': False, 'texture': <simulator.src.gym_duckietown.graphics.Texture object at 0x7f8d840777c0>, 'color': array([1, 1, 1, 

array([[[115, 209, 255],
        [115, 209, 255],
        [115, 209, 255],
        ...,
        [115, 209, 255],
        [115, 209, 255],
        [115, 209, 255]],

       [[115, 209, 255],
        [115, 209, 255],
        [115, 209, 255],
        ...,
        [115, 209, 255],
        [115, 209, 255],
        [115, 209, 255]],

       [[115, 209, 255],
        [115, 209, 255],
        [115, 209, 255],
        ...,
        [115, 209, 255],
        [115, 209, 255],
        [115, 209, 255]],

       ...,

       [[ 55,  52,  50],
        [ 55,  52,  50],
        [ 55,  52,  50],
        ...,
        [ 38,  63,  22],
        [ 53,  79,  36],
        [ 64,  89,  44]],

       [[ 55,  52,  50],
        [ 55,  52,  50],
        [ 55,  52,  50],
        ...,
        [ 31,  57,  13],
        [ 43,  69,  25],
        [ 60,  86,  40]],

       [[ 55,  52,  50],
        [ 55,  52,  50],
        [ 55,  52,  50],
        ...,
        [ 28,  54,   7],
        [ 32,  58,  12],
        [ 45,  71,  25]]

Simulate the environment. Here you can collect the state and the control inputs.

In the Duckietown environment the control input **_u_** is the steering angle, while the state **_X_** is represented by the distance from the center of the lane and the angle from the straight as shown in the image below

![controls](assets/control_model.png)



In [32]:

#
#   Simulate the environment with a PID controller 
#
speed = 0.2

DESIRED_DATAPOINTS = 1000
num_datapoints = 0

lra_pid_collector = LRA2_HELPER()

while num_datapoints < DESIRED_DATAPOINTS:
    while True:
        lane_pose = env.get_lane_pos2(env.cur_pos, env.cur_angle)
        distance_to_road_center = lane_pose.dist
        angle_from_straight_in_rads = lane_pose.angle_rad

        k_p = 10
        k_d = 1

        X = [distance_to_road_center, angle_from_straight_in_rads]
        steering =  k_p * distance_to_road_center + k_d * angle_from_straight_in_rads
        
        ### YOUR CODE HERE
        ### Collect the data here using the lra class
        lra_pid_collector.collectData(steering, X)
        num_datapoints += 1

        obs, reward, done, info = env.step([speed, steering])

        #print('Steps = %s' % (env.step_count))

    #     lra.renderEnv(env)

        if done:
            if reward < 0:
                print('*** CRASHED ***')
                env.reset()
            break

        if num_datapoints == DESIRED_DATAPOINTS:
            print('Finishing after collecting the desired number of datapoints ({})'.format(DESIRED_DATAPOINTS))
            break
            

### Don't forget to save the data!
lra_pid_collector.saveData()
os.rename('model_data.csv', 'pid_model_data.csv')

env.close()

lra.renderEnv(env)

/home/mike/duckietown/RH9/lqr-exercise/venv/lib/python3.8/site-packages/geometry/poses.py:274: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  linear = np.array(linear)
DEBUG:gym-duckietown:[3.50900775 0.         1.75486028] corresponds to tile at (5, 2) which is not drivable: {'coords': (5, 2), 'kind': 'asphalt', 'angle': 1, 'drivable': False, 'texture': <simulator.src.gym_duckietown.graphics.Texture object at 0x7f8d84001250>, 'color': array([1, 1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.0
DEBUG:gym-duckietown:pos: [3.57581015 0.         1.78895484]
DEBUG:gym-duckietown:l_pos: [3.64261255 0.         1.82304941]
DEBUG:gym-duckietown:r_pos: [3.50900775 0.         1.75486028]
DEBUG:gym-du

*** CRASHED ***


DEBUG:gym-duckietown:[1.96740205 0.         0.58320786] corresponds to tile at (3, 0) which is not drivable: {'coords': (3, 0), 'kind': 'floor', 'angle': 1, 'drivable': False, 'texture': <simulator.src.gym_duckietown.graphics.Texture object at 0x7f8d804fe880>, 'color': array([1, 1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.0
DEBUG:gym-duckietown:pos: [2.01976736 0.         0.65640536]
DEBUG:gym-duckietown:l_pos: [1.95876945 0.         0.70004312]
DEBUG:gym-duckietown:r_pos: [2.08076528 0.         0.6127676 ]
DEBUG:gym-duckietown:f_pos: [1.96740205 0.         0.58320786]
INFO:gym-duckietown: light_pos: [0.0, 3.0, 0.0, 1.0]
   ambient: instance of ndarray: array([0.25, 0.25, 0.25, 1.  ])
   diffuse: instance of ndarray: array([0.35, 0.35, 0.35, 1.  ])
  specular: instance of ndarray: array([0., 0., 0., 1.])
INFO:gym-duckietown:Starting at [1.61366413 0.         3.30376294] 0.7282206778366832


*** CRASHED ***


DEBUG:gym-duckietown:[2.34095863 0.         2.65522114] corresponds to tile at (4, 4) which is not drivable: {'coords': (4, 4), 'kind': 'asphalt', 'angle': 1, 'drivable': False, 'texture': <simulator.src.gym_duckietown.graphics.Texture object at 0x7f8def7ba7f0>, 'color': array([1, 1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.0
DEBUG:gym-duckietown:pos: [2.27378625 0.         2.71511998]
DEBUG:gym-duckietown:l_pos: [2.22387055 0.         2.659143  ]
DEBUG:gym-duckietown:r_pos: [2.32370195 0.         2.77109697]
DEBUG:gym-duckietown:f_pos: [2.34095863 0.         2.65522114]
INFO:gym-duckietown: light_pos: [0.0, 3.0, 0.0, 1.0]
   ambient: instance of ndarray: array([0.25, 0.25, 0.25, 1.  ])
   diffuse: instance of ndarray: array([0.35, 0.35, 0.35, 1.  ])
  specular: instance of ndarray: array([0., 0., 0., 1.])
INFO:gym-duckietown:Starting at [1.2293667  0.         3.14432496] 4.114446179931718


...

*** CRASHED ***


DEBUG:gym-duckietown:[1.59162826 0.         0.58181907] corresponds to tile at (2, 0) which is not drivable: {'coords': (2, 0), 'kind': 'floor', 'angle': 1, 'drivable': False, 'texture': <simulator.src.gym_duckietown.graphics.Texture object at 0x7f8d822a5640>, 'color': array([1, 1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.0
DEBUG:gym-duckietown:pos: [1.52949544 0.         0.6469306 ]
DEBUG:gym-duckietown:l_pos: [1.47523583 0.         0.59515325]
DEBUG:gym-duckietown:r_pos: [1.58375505 0.         0.69870796]
DEBUG:gym-duckietown:f_pos: [1.59162826 0.         0.58181907]
INFO:gym-duckietown: light_pos: [0.0, 3.0, 0.0, 1.0]
   ambient: instance of ndarray: array([0.25, 0.25, 0.25, 1.  ])
   diffuse: instance of ndarray: array([0.35, 0.35, 0.35, 1.  ])
  specular: instance of ndarray: array([0., 0., 0., 1.])
DEBUG:gym-duckietown:[1.3506705  0.         3.51260066] corresponds to tile at (2, 6) which is not 

*** CRASHED ***


DEBUG:gym-duckietown:[1.24154805 0.         2.92368602] corresponds to tile at (2, 4) which is not drivable: {'coords': (2, 4), 'kind': 'grass', 'angle': 1, 'drivable': False, 'texture': <simulator.src.gym_duckietown.graphics.Texture object at 0x7f8d9c03e2e0>, 'color': array([1, 1, 1, 1])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.0
DEBUG:gym-duckietown:pos: [1.17790118 0.         2.96336064]
DEBUG:gym-duckietown:l_pos: [1.1142543  0.         3.00303525]
DEBUG:gym-duckietown:r_pos: [1.24154805 0.         2.92368602]
DEBUG:gym-duckietown:f_pos: [1.13029164 0.         2.88698439]
INFO:gym-duckietown: light_pos: [0.0, 3.0, 0.0, 1.0]
   ambient: instance of ndarray: array([0.25, 0.25, 0.25, 1.  ])
   diffuse: instance of ndarray: array([0.35, 0.35, 0.35, 1.  ])
  specular: instance of ndarray: array([0., 0., 0., 1.])
INFO:gym-duckietown:Starting at [1.47149548 0.         2.11876741] 5.885370644726965


*** CRASHED ***
Finishing after collecting the desired number of datapoints (1000)


In the cell below run a random controll inputs and collect the data as in the previous exercise 

In [ ]:

#
#   Simulate the environment with a random controller 
#
speed = 0.2
while True:
    lane_pose = env.get_lane_pos2(env.cur_pos, env.cur_angle)
    distance_to_road_center = lane_pose.dist
    angle_from_straight_in_rads = lane_pose.angle_rad

    X = [distance_to_road_center, angle_from_straight_in_rads]

    ### YOUR CODE HERE
    ### Collect the data here using the lra class
    steering = 


    obs, reward, done, info = env.step([speed, steering])

    #print('Steps = %s' % (env.step_count))

    

    if done:
        if reward < 0:
            print('*** CRASHED ***')
        break


### Don't forget to save the data!

env.close()

lra.renderEnv(env)

# Model estimation and Data cleaning


In [ ]:
from sklearn.linear_model import Ridge
import pandas as pd

You are asked to estimate the dynamics model of a Duckiebot. We can approximate the dynamics of a Duckiebot with a linear system:

$$
x(t+1) = A \cdot x(t) + B \cdot u(t)
$$


The goal here is to guess the values of the matrices A and B.

_Hint: linear regression problem_


Divide your dataset in test and training:

In [ ]:
### Clean the data before splitting the dataset

data = lra.loadData("model_data.csv")



Train the linear model on the train set:

In [ ]:


##  YOR CODE HERE



Check how good is the model you got according to your test set.

# LQR design

Change the **_A_** and **_B_** matrices with the model you found the previous section. Then try to tune the **_Q_** and **_R_** matrices in order to have good driving performance.

The function LQRGain returns the **_K_** gain needed to realize the state feedback controller.

$\color{red}{\text{DO NOT CHANGE THE NAME OF THIS FUNCTION.}}$


In [ ]:

#
#   Find the state feedback control gain
#
def LQRGain():
    from lraClass import LRA2_HELPER
    import numpy as np
    
    lra = LRA2_HELPER()

    ### Write the A and B matrices you found in the previous section.
    A = np.array([[0.0, 0.0 ],
            [0.0, 0.0 ]])

    B = np.array([[0.0, 0.0]]).T

    Q = np.array([[0.0, 0.0], [0.0, 0.0]])
    R = np.array([[0.0]])


    # Solve Algebric Riccati Equation
    S = lra.solveRiccati(A,B,Q,R)

    # find the gain K
    K = 

    return K


In [ ]:

speed=0.2


K=LQRGain()

env = lra.wrapEnv(env)
env.reset()

#
#   Simulate the environment
#
while True:
    lane_pose = env.get_lane_pos2(env.cur_pos, env.cur_angle)
    distance_to_road_center = lane_pose.dist
    angle_from_straight_in_rads = lane_pose.angle_rad

    X = [distance_to_road_center, angle_from_straight_in_rads]

    # YOUR CODE HERE
    # find the steering using the gain you found.
    steering = 
    
    obs, reward, done, info = env.step([speed, steering])

    #print('Steps = %s' % (env.step_count))

    if done:
        if reward < 0:
            print('*** CRASHED ***')
        break

env.close()

lra.renderEnv(env)